# Notebook for Creating Figures from OpenFOAM Solutions

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv

## Define Helper Files

In [2]:
def camera_settings(PYLON=False):
    """ Define settings for viewing the figures """
    if PYLON is True:
        camera = pv.Camera()
        camera.position = (15.9143, 3.83627, 28.3582)
        camera.focal_point = (15.9143, 3.83627, 13.75)
        camera.view_angle = 30
    else:
        camera = pv.Camera()
        camera.position = (15.9919, 3.10319, 67.1243)
        camera.focal_point = (15.9919, 3.10319, -5.08432e-08)
        camera.view_angle = 30
    
    return camera

In [3]:
def slice_center(mesh, NORMAL='z', ORIGIN=[0, 0, 0], translate=False, CONTOUR=False):
    """Slice mesh through center in normal direction, move to zero normal."""
    slice_mesh = mesh.slice(normal=NORMAL, origin=ORIGIN, contour=CONTOUR)
    if translate is not False:
        if NORMAL == 'z':
            slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
        elif NORMAL == 'x':
            slice_mesh.translate((-slice.mesh.center[0], 0, 0), inplace=True)
        elif NORMAL == 'y':
            slice_mesh.translate((0, -slice.mesh.center[1], 0), inplace=True)
        else:
            print('ERROR: Normal not found - ', NORMAL)
            slice_mesh = None
    return slice_mesh

In [4]:
def read_mesh(nfile):
    """ Read the OpenFOAM solution, needs the file generated by paraFoam (e.g. *.OpenFOAM) """
    # Read the reference file created by openFOAM `parafoam`
    reader = pv.POpenFOAMReader(nfile)
    # set the active time 
    reader.set_active_time_value(reader.time_values[-1])
    reader.cell_to_point_creation = True
    # Define the mesh
    mesh = reader.read()
    # Define the internal mesh and boundaries
    internal = mesh["internalMesh"]
    boundaries = mesh["boundary"]
    
    return internal, boundaries

In [5]:
def sample_line(mesh, pointA, pointB, field=None):
    """ Sample fields across a line, marked by two points, within the mesh """
    # Sample across the mesh designated by inputed points
    line = mesh.sample_over_line(pointA,
                                 pointB,
                                 resolution=100,
                                 tolerance=0.2,
                                 )
    # Define the distance of this line
    distance = line['Distance']
    
    # Check to see if a desired field to extract
    if field is not None:
        subset = line.get_array(field)
    else:
        subset = None
    
    if field is not None:
        return distance, subset
    else:
        return distance, line

In [6]:
def plot_field(ndir, 
               nfile,
               FIELD='U', 
               RANGE=[95, 145], 
               CONTOUR=True, 
               ORIGIN=[0, 0, 0], 
               NORMAL='z', 
               CMAP='bwr',
               SAVE=False,
               PYLON=False,
               LINE=False):
    """ 
    Create 2-D Figure of the OpenFOAM internal mesh for selected field. 
    Normal decides which plane to exclude from display.
    Origin moves where the slice is preformed.
    """
    # add extension to the input directory file to reference *.OpenFOAM files
    solution = ndir + nfile['filename'] + nfile['filename'][:-1] + '.OpenFoam'
    
    # Define label for input field
    LABEL = {"U" : "Velocity [m/s]",
             "p" : "Pressure [Pa]", 
             "T" : "Temperature [C]"
            }
    
    # Read the Mesh
    internal, boundaries = read_mesh(solution)
    
    # Slice the Mesh
    if PYLON is True:
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=nfile['pylon'])
    else:
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=ORIGIN)
    
    # Contour the Mesh
    contours = p3_slice.contour(scalars=FIELD, isosurfaces=15, rng=RANGE)
    
    # Set a custom position and size
    sargs = dict(height=0.10, 
                 vertical=False, 
                 position_x=0.2, 
                 position_y=0.05, 
                 fmt="%.0f", 
                 color='black', 
                 n_labels=6, 
                 title=LABEL[FIELD])
    
    # Create the plotter
    pt = pv.Plotter()
    # Add the slice mesh
    pt.add_mesh(p3_slice, 
                scalars=FIELD, 
                preference='point', 
                cmap=CMAP, 
                clim=RANGE,
                scalar_bar_args=sargs)
    
    # Add the contours
    pt.add_mesh(contours, 
                color='white', 
                preference='cell', 
                line_width=1)
    
    # Add a ruler
    if PYLON is True:
        pt.add_ruler(pointa=[10, -0.35, 0.0],
                     pointb=[20, -0.35, 0.0],
                     title='Distance [m]',
                     tick_color='black',
                     label_color='black'
                    )
    else:
        pt.add_ruler(pointa=[-5, -5, 0.0],
                     pointb=[35, -5, 0.0],
                     title='Distance [m]',
                     tick_color='black',
                     label_color='black'
                    )
   
    # Add a title
    pt.add_title(nfile['filename'][:-1], 
                 font='courier', 
                 color='k', 
                 font_size=10
                )
    
    # Define the camera view
    if PYLON is True:
        pt.camera = camera_settings(PYLON=True)
    else:
        pt.camera = camera_settings()
    
    # Save the figure
    if SAVE is True:
        # Define a figure title
        nout = nfile['filename'][:-1] + '_' + str(FIELD) + '.svg'
        # Save the figure
        pt.save_graphic(nout, raster=True)
        
    if LINE is True:
        # Make two points to construct the line between
        A = [13.5, nfile['pylon'][1], nfile['pylon'][2]]
        B = [14.5, nfile['pylon'][1], nfile['pylon'][2]]

        # Preview how this line intersects this mesh
        line = pv.Line(A, B)
        pt.add_mesh(line, color="white", line_width=10)
        
    # Display the figure
    pt.show()
    
    # Free up some memory
    del internal, p3_slice, contours, boundaries, pt

In [7]:
def plot_line(ndir,
              nfiles,
              FIELD='U',
              reference=False,
              NORMAL='z',
              ORIGIN=[0,0,0],
              **kwargs):
    """
    Create line plot for selected pylon against all angles of attack and reference solutions
    
    Parameters
    ----------
    ndir = str
        Directory holding these solutions
    nfiles = dict
        Dictionary holding pylon configuration directory names, CDP location, etc
    FIELD = str
        Scalar to display
    reference = boolean, Default=False
        If True, compare solutions against No-Pylon solutions
        
    Returns
    -------
    fig - figure plotting sampled line
    """
     # Define label for input field
    LABEL = {"U" : "Velocity [m/s]",
             "p" : "Pressure [Pa]", 
             "T" : "Temperature [C]"
            }
    
    # define a blank dictionary to hold the sample over line
    solutions = {}
    print('nfiles', nfiles)
    for case in nfiles:
        print('in nfiles:', case)
        # add extension to the input directory file to reference *.OpenFOAM files
        directory = ndir + case['filename'] + case['filename'][:-1] + '.OpenFoam'
    
        # Read the Mesh
        internal, boundaries = read_mesh(directory)
    
        # Slice the Mesh
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=case['pylon'])
            
        # Sample over line
        disx, field = sample_line(p3_slice, case['pylon'], case['end'])
        
        solutions[case['tag']] = {'distance' : disx,
                           'scalar' : field
                          }
        # clear up some memory
        del internal, boundaries, p3_slice, disx, field
        
    # if reference is set, also extract the no pylon solutions
    # COMING SOON!
    
    # Display
    fig, axarr = plt.subplots(1, 1, figsize=[12, 8])
    for case in solutions:
        if FIELD == 'U':
            axarr.plot(solutions[case]['distance']-.922, 
                       solutions[case]['scalar'].point_data['U'][:, 0], 
                       label=case + '_Ux')
            axarr.plot(solutions[case]['distance']-.922, 
                       solutions[case]['scalar'].point_data['U'][:, 1],
                       linestyle='--',
                       label=case + '_Uy')
            axarr.plot(solutions[case]['distance']-.922, 
                       solutions[case]['scalar'].point_data['U'][:, 2], 
                       linestyle='-.',
                       label=case + '_Uz')
        else:
            axarr.plot(solutions[case]['distance']-.922,
                       solutions[case]['scalar'],
                       label=case + FIELD)
        axarr.set_xlabel('Distance from Cloud Droplet Probe Sample Volume [m]')
        axarr.set_ylabel('Velocity [m/s]')
        axarr.grid(True)
        plt.legend(loc='lower left')

## Define the Directory, Case Structure, Fields and Pylon Locations

In [20]:
# Define the directory hosting the solution directories
solution_dir = "/Users/jrobrien/Dissertation/data/solutions/"
# Define available solution directories
case_120 = {'navy_120_aoa0' : {'filename' : 'NASA_navyPylon_v2_tas120_aoa0_900T33/',
                               'pylon' : [13.5, 3.05, 13.75],
                               'tag' : 'Navy120_AoA0'
                              },
            'extend_120_aoa0' : {'filename' : 'NASA_extendedPylon_v2_tas120_aoa0_900T33/',
                                 'pylon' : [13.0, 2.725, 13.7],
                                 'end' : [13.92, 2.725, 13.7],
                                 'tag' : 'Extend120_AoA0',
                                },
            'no_120_aoa0' : {'filename' : 'NASA_noPylons_v2_tas120_aoa0_900T33/',
                             'pylon' : [0, 2.75, 13.7],
                             'tag' : 'Null120_AoA0',
                            },
            'navy_120_aoaPos4' : {'filename' : 'NASA_navyPylon_v3_tas120_aoaPos4_900T33/',
                                  'pylon' : [0, 2.75, 13.75],
                                  'tag' : 'Navy120_AoA+4',
                                 },
            'extend_120_aoaPos4' : {'filename' : 'NASA_extendedPylon_v3_tas120_aoaPos4_900T33/',
                                    'pylon' : [13.0, 2.725, 13.7],
                                    'end' : [13.92, 2.725, 13.7],
                                    'tag' : 'Extend120_AoA+4',
                                   },
            'no_120_aoaPos4' : {'filename' : 'NASA_noPylons_v3_tas120_aoaPos4_900T33/',
                                'pylon' : [0, 2.75, 13.7],
                                'tag' : 'Null120_AoA+4',
                               },
            'navy_120_aoaNeg4' : {'filename' : 'NASA_navyPylon_v3_tas120_aoaNeg4_900T33/',
                                  'pylon' : [0, 2.75, 13.75],
                                  'tag' : 'Navy120_AoA-4',
                                 },
            'extend_120_aoaNeg4' : {'filename' : 'NASA_extendedPylon_v3_tas120_aoaNeg4_900T33/',
                                    'pylon' : [13.0, 2.725, 13.7],
                                    'end' : [13.92, 2.725, 13.7],
                                    'tag' : 'Extend120_AoA-4',
                                   },
            'no_120_aoaNeg4' : {'filename' : 'NASA_noPylons_v3_tas120_aoaNeg4_900T33/',
                                'pylon' : [0, 2.75, 13.7],
                                'tag' : 'Null120_AoA-4'
                               }
           }
# Define available solution directories
case_140 = {'navy_140_aoa0' : {'filename' : 'NASA_navyPylon_v2_tas140_aoa0_800T20/',
                               'pylon' : [0, 2.75, 13.75]
                              },
            'extend_140_aoa0' : {'filename' : 'NASA_extendedPylon_v2_tas140_aoa0_800T20/',
                                 'pylon' : [13.0, 2.725, 13.7],
                                 'end' : [13.92, 2.725, 13.7]
                                },
            'no_140_aoa0' : {'filename' : 'NASA_noPylons_v2_tas140_aoa0_800T20/',
                             'pylon' : [0, 2.75, 13.7]
                            },
            'navy_140_aoaPos4' : {'filename' : 'NASA_navyPylon_v3_tas140_aoaPos4_800T20/',
                                  'pylon' : [0, 2.75, 13.75]
                                 },
            'extend_140_aoaPos4' : {'filename' : 'NASA_extendedPylon_v2_tas140_aoaPos4_800T20/',
                                    'pylon' : [13.0, 2.725, 13.7],
                                    'end' : [13.92, 2.725, 13.7]
                                   },
            'no_140_aoaPos4' : {'filename' : 'NASA_noPylons_v2_tas140_aoaPos4_800T20/',
                                'pylon' : [0, 2.75, 13.7]
                               },
            'navy_140_aoaNeg4' : {'filename' : 'NASA_navyPylon_v2_tas140_aoaNeg4_800T20/',
                                  'pylon' : [0, 2.75, 13.75]
                                 },
            'extend_140_aoaNeg4' : {'filename' : 'NASA_extendedPylon_v2_tas140_aoaNeg4_800T20/',
                                    'pylon' : [13.0, 2.725, 13.7],
                                    'end' : [13.92, 2.725, 13.7]
                                   },
            'no_140_aoaNeg4' : {'filename' : 'NASA_noPylons_v2_tas140_aoaNeg4_800T20/',
                                'pylon' : [0, 2.75, 13.7]
                               }
           }

In [9]:
scalar_field = {'U' : {'range' : [95, 145]},
                'T' : {'range' : [295, 318]},
                'p' : {'range' : [89000, 91000]}
               }

In [10]:
selected_line = [case_120['extend_120_aoa0'], case_120['extend_120_aoaPos4'], case_120['extend_120_aoaNeg4']]

## Process the OpenFOAM Solutions

In [21]:
# Process the selected Scalar fields
for case in case_120:
    if case == 'navy_120_aoa0':
        for scalar in scalar_field:
            if scalar == 'U':
                print(case_120[case])
                plot_field(solution_dir, 
                           case_120[case], 
                           SAVE=False, 
                           FIELD=scalar, 
                           RANGE=scalar_field[scalar]['range'], 
                           PYLON=True,
                           LINE=True)

{'filename': 'NASA_navyPylon_v2_tas120_aoa0_900T33/', 'pylon': [0, 3.05, 13.75], 'tag': 'Navy120_AoA0'}


2023-06-02 23:32:44.773 ( 312.168s) [           4E5D3]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Widget(value="<iframe src='http://localhost:50319/index.html?ui=P_0x2e331d280_4&reconnect=auto' style='width: …

In [ ]:
# Process the selected Scalar fields over the line
plot_line(solution_dir, 
          selected_line, 
          FIELD='U')